# **Importation des bibliothèques**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows',None)

In [3]:
pd.set_option('display.max_columns',None)

# **charger les données**

In [4]:
df=pd.read_csv('/content/train (1).csv')

In [5]:
df.head(15)

,id,person_description,yearly_income,customer_order,gross_weight,net_weight,is_recyclable,additional_features_in_market,promotion_name,store_kind,store_sales,store_cost,grocery_area,frozen_area,meat_area,cost,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,review_score,store_area,package_weight
0,mc_ID_1,Single Female with three children and educatio...,€42K+,"Snack Foods from Snack Foods department, Order...",16.571,14.972,non recyclable,NaN,Budget Bargains,Supermarket,$6.36 Millions,1.9716 Millions,2049.72,457.36,NaN,708.665,abaete,MG,-19.16050583709177,-45.45377571289521,betim,MG,-19.95866259017359,-44.19878943272789,4.0,NaN,NaN
1,mc_ID_2,"MaRrIeD MaLe wItH TwO ChIlDrEn,eDuCaTiOn: HiGh...",30K+,"Magazines from Periodicals department, Ordered...",-28.6358,data: 27.1822,recyclable,['Florist'],Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,1322.21,523.32,348.85,val: 564.2647,Sao PauLo,Sp,data: -23.50609758451153,-46.73241237719997,PRAIE GRANDE,$P,-24.026786758814687,-46.49304889148245,5.0,NaN,NaN
2,mc_ID_3,"Single Female with three children, education: ...",30K+ annually,"Dairy from Dairy department, Ordered Brand : C...",14.2161,amount: 11.2944,recyclable,['Coffee Bar'],Savings Galore,NaN,1.98 Millions,0.6732 Millions,NaN,440.92,293.95,724.5119,sao paulo,SP,-23.665780780743223,-46.6821276135092,Sao Paulo,SP,-23.69289565500035,-46.68021046531817,1.0,NaN,NaN
3,mc_ID_5,"Single Male with four children, education: hig...",30K+,"Candles from Household department, Ordered Bra...",15.4081,13.9545,recyclable,['Coffee Bar'],Weekend Discount,Supermarket,5.22 Millions,1.6182 Millions,1236.07,440.92,293.95,364.1649,mogi-guacu,5P,-22.382941116125448,-46.94664125419024,ibitinga,SP,-21.7664768468313,-48.83154737836604,5.0,NaN,NaN
4,mc_ID_7,"Married Male with four children, education: pa...",10K+,"Vegetables from Frozen Foods department, Order...",17.2978,-14.376,recyclable,NaN,Shelf Clearing Days,SUpermArKET,6.57 Millions,2.6937 Millions,1623.48,341.05,NaN,288.8006,embu-guacu,SP,-23.82852167738724,-46.814795982633896,atibaia,SP,-23.112774381969185,-46.5488845105504,5.0,NaN,NaN
5,mc_ID_8,"Married Male with five children, education: hi...",30K+,Beer and Wine from Alcoholic Beverages departm...,25.1472,23.5482,recyclable,"['Bar For Salad', 'Ready Food']",Big Time Discounts,sUPermARkEt,2.43 Millions,1.1907 Millions,2449.91,472.27,313.55,501.9754,pocos de caldas,mg,data: -21.79104335195045,-46.56244368105504,sao paulo_,SP,-23.510081986121538,amount: -46.65907867000398,measure: 5.0,NaN,NaN
6,mc_ID_10,"Married Male with three children, education: H...",30K+ annually,"Frozen Desserts from Frozen Foods department, ...",24.7111,21.8039,non recyclable,"['Ready Food', 'Coffee Bar', 'Video Store', 'F...",Big Time Discounts,Deluxe,3.96 Millions,1.7028 Millions,1734.5,503.07,335.38,583.1973,pEdRo leoPOlDO,MG,val: -19.652292291953287,-44.056767929538886,PETROPOLIS,_RJ,-22.549033360244323,-43.210094724044055,5.0,NaN,NaN
7,mc_ID_11,"Single Female with two children, education: pa...",10K+,"Breakfast Foods from Frozen Foods department, ...",11.1054,8.1837,non recyclable,NaN,Big Time Discounts,$UPERMARKET,1.71 Millions,0.513 Millions,1508,227.8,NaN,574.9339,sabara,MG,-19.862833868691293,data: -43.85726075973933,borda da mata,MG,-22.264093569966427,-46.15856420525942,4.0,NaN,NaN
8,mc_ID_12,"Married Female with No kids, education: high s...",50K+,"Bread from Baked Goods department, Ordered Bra...",27.4729,24.4204,recyclable,['Coffee Bar'],Weekend Discount,SUPERMARKET,$8.79 Millions,3.7797 Millions,1236.07,440.92,293.69,364.1649,sao paulo,SP,-23.560566359426364,-46.65761042867444,SERRANA,Sp,-21.21594896945095,-47.59002511568736,5.0,NaN,NaN
9,mc_ID_13,"Married Female with four children, education: ...",70 Thousand,"Baking Goods from Baking Goods department, Ord...",22.8214,amount: 21.2225,recyclable,"['Coffee Bar', 'Bar For Salad', 'Ready Food', ...",Pick Your Savings,Deluxe,6.78 Millions,3.1188 Millio

# **Nettoyage des données**

In [6]:
def clean_yearly_income(val):
    if pd.isna(val):
        return np.nan

    val = str(val).lower().strip()

    # valeurs non exploitables
    invalid = [
        'not available', 'data missing', 'missing', 'unknown',
        'tbd', 'nil', '...', '###', '--', '?', 'not specified'
    ]
    if any(x in val for x in invalid):
        return np.nan

    # valeurs négatives / codes erreur
    if val in ['-1', '-999']:
        return np.nan

    # monthly income → annual
    if 'monthly' in val:
        num = re.search(r'(\d+(?:\.\d+)?)', val)
        if num:
            return float(num.group(1)) * 12
        return np.nan

    # extraire nombre
    num = re.search(r'(\d+(?:\.\d+)?)', val)
    if not num:
        return np.nan

    value = float(num.group(1))

    # milliers (K, thousand)
    if 'k' in val or 'thousand' in val:
        return value * 1000

    # sinon valeur déjà annuelle
    return value


In [7]:
df['yearly_income'] = df['yearly_income'].apply(clean_yearly_income)


In [8]:
def clean_is_recyclable(value):
    if pd.isna(value):
        return np.nan

    value = str(value).lower().strip()

    # valeurs manquantes déguisées
    missing_values = [
        'not specified', 'data missing', 'missing', 'unknown',
        'nil', 'tbd', 'to be determined', '...', '###',
        '--', '?', '-999', '-1'
    ]
    if value in missing_values:
        return np.nan

    # normalisation
    value = re.sub(r'[().]', '', value)

    if value in ['yes', 'recyclable']:
        return 'recyclable'

    if value in ['no', 'non recyclable']:
        return 'non recyclable'

    return np.nan


In [9]:
df['is_recyclable'] = df['is_recyclable'].apply(clean_is_recyclable)


In [10]:
def extract_weight_to_kg(value):
    if pd.isna(value):
        return pd.NA

    value = str(value).lower()

    # Extraire le premier nombre (positif ou négatif)
    match = re.search(r'-?\d+\.?\d*', value)
    if not match:
        return pd.NA

    weight = float(match.group())

    # Nettoyage des valeurs aberrantes
    if weight < 0:
        return pd.NA

    # Détection de l’unité
    if 'gram' in value or 'g' in value:
        return weight / 1000

    elif 'lb' in value:
        return weight * 0.453592

    else:
        # par défaut : kg
        return weight


In [11]:
df['gross_weight'] = df['gross_weight'].apply(extract_weight_to_kg)
df['net_weight']   = df['net_weight'].apply(extract_weight_to_kg)


In [12]:
def normalize_weight_to_kg(x):
    if pd.isna(x):
        return np.nan

    x = float(x)

    if x < 1:
        return x * 1000          # tonnes → kg
    elif x < 100:
        return x                 # déjà kg
    elif x < 10000:
        return x / 1000          # grammes → kg
    else:
        return x / 1000          # grammes → kg

df['gross_weight'] = df['gross_weight'].apply(normalize_weight_to_kg)
df['net_weight'] = df['net_weight'].apply(normalize_weight_to_kg)


In [13]:
def clean_promotion_name(val):
    if pd.isna(val):
        return pd.NA

    val = str(val).strip().lower()

    # Valeurs manquantes
    missing_values = {
        'nil', '?', '-999', '...', 'unknown', 'tbd', '-1', '--', '###',
        'missing', 'not available', 'not specified', 'data missing',
        'to be determined'
    }

    if val in missing_values:
        return pd.NA

    # Nettoyage texte
    val = re.sub(r'[().]', '', val)   # enlever (), .
    val = re.sub(r'\s+', ' ', val)    # espaces multiples
    val = val.strip()

    # Normalisation de quelques promotions clés
    mapping = {
        'three for one': 'Three For One',
        'two for one': 'Two For One',
        'sale : double down': 'Sale: Double Down',
        'roller savings high': 'Roller Savings (High)',
        'budget bargains': 'Budget Bargains',
        'shelf emptiers': 'Shelf Emptiers',
        'shelf clearing days': 'Shelf Clearing Days',
        'weekend discount': 'Weekend Discount',
        'save price': 'Save Price',
        'money savers': 'Money Savers',
        'super duper savers': 'Super Duper Savers',
        'you save days': 'You Save Days',
        'price winners': 'Price Winners',
        'discount frenzy': 'Discount Frenzy',
        'tip top savings': 'Tip Top Savings'
    }

    return mapping.get(val, val.title())

df['promotion_name'] = df['promotion_name'].apply(clean_promotion_name)


In [14]:
def clean_store_kind(val):
    if pd.isna(val):
        return "Unknown"

    # nettoyage de base
    v = str(val).strip().lower()

    # normalisation chiffres → lettres
    v = (
        v.replace('0', 'o')
         .replace('1', 'l')
         .replace('3', 'e')
         .replace('5', 's')
         .replace('7', 't')
         .replace('4', 'a')
    )

    # suppression caractères spéciaux
    v = re.sub(r'[^a-z\s\-]', '', v)

    # règles métier
    if re.search(r'super\s*market|supermarket', v):
        return "Supermarket"

    if re.search(r'deluxe|daluxe|delux', v):
        return "Deluxe"

    if re.search(r'gourmet|gourmat|gourm', v):
        return "Gourmet"

    if re.search(r'mid\s*-\s*size|midsize|mid\s*size', v):
        return "Mid-Size"

    if re.search(r'small\s*grocery|smallgrocery|groc', v):
        return "Small Grocery"

    return "Unknown"

df['store_kind'] = df['store_kind'].apply(clean_store_kind)


In [15]:
def clean_store_sales(val):
    if pd.isna(val):
        return np.nan

    v = str(val).lower().strip()

    # valeurs non exploitables
    if v in ['-1', 'missing', '###', 'unknown', 'nan', '']:
        return np.nan

    # suppression devises et espaces inutiles
    v = re.sub(r'[€$,]', '', v)

    # détecter millions
    is_million = 'million' in v or 'm' in v

    # extraire la partie numérique
    match = re.search(r'([\d]+(?:\.\d+)?)', v)
    if not match:
        return np.nan

    value = float(match.group(1))

    # appliquer le bon facteur
    if is_million:
        value *= 1_000_000

    return value

df['store_sales'] = df['store_sales'].apply(clean_store_sales)



In [16]:
df.loc[df['store_sales'] < 1_000, 'store_sales'] *= 1_000_000


In [17]:
df['store_cost'] = df['store_cost'].apply(clean_store_sales)

In [18]:
df.loc[df['store_cost'] < 1_000, 'store_cost'] *= 1_000_000

In [19]:
df['store_cost'].describe()

,store_cost
count,2.848400e+04
mean,2.720564e+06
std,1.032775e+07
min,1.632000e+05
25%,1.490250e+06
50%,2.382800e+06
75%,3.492900e+06
max,9.990000e+08


In [20]:
def clean_grocery_area(val):
    if pd.isna(val):
        return np.nan

    v = str(val).strip()

    # valeurs non exploitables
    if v in ['...', 'nan', 'NaN', '', '--']:
        return np.nan

    # supprimer parenthèses
    v = re.sub(r'[()]', '', v)

    # supprimer point final multiple
    v = re.sub(r'\.+$', '', v)

    try:
        return float(v)
    except ValueError:
        return np.nan

df['grocery_area'] = df['grocery_area'].apply(clean_grocery_area)


In [21]:
df['frozen_area'] = df['frozen_area'].apply(clean_grocery_area)

In [22]:
df['meat_area'] = df['meat_area'].apply(clean_grocery_area)

In [23]:
def clean_cost(val):
    if pd.isna(val):
        return np.nan

    v = str(val).lower().strip()

    # valeurs non exploitables
    if v in ['...', 'nan', '', '--', 'missing', 'unknown', '-1', '-999']:
        return np.nan

    # suppression devise
    v = re.sub(r'[$€]', '', v)

    # extraction du nombre
    match = re.search(r'-?\d+\.?\d*', v)

    if match:
        return float(match.group())

    return np.nan

df['cost'] = df['cost'].apply(clean_cost)


In [24]:
import unicodedata

def clean_city_name(city):
    if pd.isna(city):
        return np.nan

    # 1. minuscules
    city = city.lower()

    # 2. supprimer accents
    city = unicodedata.normalize('NFKD', city)
    city = city.encode('ascii', 'ignore').decode('utf-8')

    # 3. remplacer erreurs fréquentes
    replacements = {
        '0': 'o',
        '1': 'i',
        '4': 'a',
        '5': 's',
        '@': 'a',
        '$': 's'
    }
    for k, v in replacements.items():
        city = city.replace(k, v)

    # 4. supprimer tabulations, retours ligne
    city = re.sub(r'[\t\n\r]', ' ', city)

    # 5. supprimer caractères non alphabétiques
    city = re.sub(r'[^a-z\s]', '', city)

    # 6. supprimer espaces multiples
    city = re.sub(r'\s+', ' ', city).strip()

    return city

In [25]:
df['customer_city'] = df['customer_city'].apply(clean_city_name)


In [26]:
VALID_STATES = {
    'AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS',
    'MG','PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC',
    'SE','SP','TO'
}

def clean_customer_state(val):
    if pd.isna(val):
        return np.nan

    v = str(val).upper()

    # supprimer tabs, retours, underscores
    v = re.sub(r'[\t\n_]', '', v)

    # corriger substitutions fréquentes
    v = (
        v.replace('0', 'O')
         .replace('5', 'S')
         .replace('1', 'I')
         .replace('@', 'A')
         .replace('$', 'S')
    )

    # garder uniquement lettres
    v = re.sub(r'[^A-Z]', '', v)

    # validation métier
    if v in VALID_STATES:
        return v

    return np.nan

df['customer_state'] = df['customer_state'].apply(clean_customer_state)


In [27]:
def clean_latitude(val):
    if pd.isna(val):
        return np.nan

    text = str(val)

    # regex : nombre décimal signé
    match = re.search(r'-?\d+(?:\.\d+)?', text)

    if not match:
        return np.nan

    lat = float(match.group())

    # validation latitude
    if -90 <= lat <= 90:
        return lat

    return np.nan

df['customer_lat'] = df['customer_lat'].apply(clean_latitude)


In [28]:
df['customer_lng'] = df['customer_lng'].apply(clean_latitude)

In [29]:
def normalize_city(city):
    if pd.isna(city):
        return None

    city = str(city).strip().lower()

    # supprimer accents
    city = unicodedata.normalize('NFKD', city)
    city = city.encode('ascii', 'ignore').decode('utf-8')

    # supprimer caractères parasites
    city = re.sub(r'[^a-z\s]', ' ', city)

    # supprimer espaces multiples
    city = re.sub(r'\s+', ' ', city).strip()

    # valeurs non valides
    if city in ['unknown', 'tbd', 'not specified', 'data missing', 'missing', 'nil', '']:
        return None

    return city

df['seller_city'] = df['seller_city'].apply(normalize_city)


In [30]:
mask_sp = df['seller_city'].str.contains(
    r'\b(paulo|paul|paul[oau]|opaulo|saopaulo)\b',
    na=False
)

df.loc[mask_sp, 'seller_city'] = 'sao paulo'


In [31]:
CITY_RULES = {
    'ibitinga': r'ibit',
    'curitiba': r'curit',
    'belo horizonte': r'horiz',
    'rio de janeiro': r'janeir',
    'guarulhos': r'guarul',
    'campinas': r'campin',
}

for city, pattern in CITY_RULES.items():
    mask = df['seller_city'].str.contains(pattern, na=False)
    df.loc[mask, 'seller_city'] = city


In [32]:
def normalize_city(text):
    if pd.isna(text):
        return None

    # convertir en string
    text = str(text)

    # minuscules
    text = text.lower()

    # supprimer accents (é → e, ç → c)
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))

    # remplacer chiffres/symboles fréquents
    text = text.replace('0', 'o')
    text = text.replace('1', 'l')
    text = re.sub(r'[^a-z\s]', ' ', text)

    # supprimer espaces multiples
    text = re.sub(r'\s+', ' ', text).strip()

    return text.upper()

df['seller_city'] = df['seller_city'].apply(normalize_city)


In [33]:
city_mapping = {
    'SAO PAULO': 'SAO PAULO',
    'SAO PAOLO': 'SAO PAULO',
    'SAO PAULOO': 'SAO PAULO',
    'RIO JANEIRO': 'RIO DE JANEIRO',
    'BELO HORIZONTE': 'BELO HORIZONTE',
}

df['seller_city'] = df['seller_city'].replace(city_mapping)


In [34]:
VALID_STATES = {
    'AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS',
    'MG','PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC',
    'SP','SE','TO'
}
def clean_state(val):
    if pd.isna(val):
        return None

    val = str(val).upper()

    # corrections fréquentes
    val = val.replace('$', 'S').replace('5', 'S').replace('0', 'O')

    # supprimer tout sauf lettres
    val = re.sub(r'[^A-Z]', '', val)

    # garder uniquement les 2 premières lettres
    val = val[:2]

    # validation finale
    if val in VALID_STATES:
        return val
    else:
        return None

df['seller_state'] = df['seller_state'].apply(clean_state)


In [35]:
df['seller_lat'] = df['seller_lat'].apply(clean_latitude)

In [36]:
df['seller_lng'] = df['seller_lng'].apply(clean_latitude)

In [37]:
def clean_review_score(val):
    if pd.isna(val):
        return np.nan

    val = str(val).lower().strip()

    # valeurs invalides
    if val in ['nan', 'none', 'unknown', 'missing', 'not specified',
               'not available', 'tbd', '--', '###', '-999', '?', 'nil']:
        return np.nan

    # pourcentage (ex: 100%)
    perc = re.search(r'(\d+(?:\.\d+)?)\s*%', val)
    if perc:
        return round(float(perc.group(1)) * 5 / 100, 1)

    # format x/10
    frac10 = re.search(r'(\d+(?:\.\d+)?)\s*/\s*10', val)
    if frac10:
        return round(float(frac10.group(1)) * 5 / 10, 1)

    # format x/5
    frac5 = re.search(r'(\d+(?:\.\d+)?)\s*/\s*5', val)
    if frac5:
        return float(frac5.group(1))

    # extraire nombre simple (Rating, stars, amount, data, val, measure)
    num = re.search(r'(\d+(?:\.\d+)?)', val)
    if num:
        score = float(num.group(1))
        if 0 < score <= 5:
            return score

    return np.nan

df['review_score'] = df['review_score'].apply(clean_review_score)


> extarct les variable possible dans le varibale **"person_description"** et **"parse_customer_order"**



In [38]:
def normalize_text(val):
    if pd.isna(val):
        return None

    val = val.lower()

    # corrections fréquentes
    val = val.replace('kidren', 'children').replace('childran', 'children')

    # supprimer ponctuation excessive
    val = re.sub(r'[.,]', ' ', val)

    # supprimer espaces multiples
    val = re.sub(r'\s+', ' ', val).strip()

    return val

df['person_description'] = df['person_description'].apply(normalize_text)


In [39]:
#Statut marital
def extract_marital(val):
    if val is None:
        return None
    if 'single' in val:
        return 'Single'
    if 'married' in val:
        return 'Married'
    return None

In [40]:
#gender
def extract_gender(val):
    if val is None:
        return None
    if 'female' in val:
        return 'Female'
    if 'male' in val:
        return 'Male'
    return None


In [41]:
#nombre d'enfant
def extract_children(val):
    if val is None:
        return None

    mapping = {
        'no children': 0,
        'one children': 1,
        'two children': 2,
        'three children': 3,
        'four children': 4
    }

    for k, v in mapping.items():
        if k in val:
            return v

    return None


In [42]:
#niveau d'etude
def extract_education(val):
    if val is None:
        return None

    if 'bachelors' in val:
        return 'Bachelors'
    if 'partial college' in val:
        return 'Partial College'
    if 'partial high school' in val:
        return 'Partial High School'
    if 'high school degree' in val:
        return 'High School'

    return None


In [43]:
#role job
def extract_job(val):
    if val is None:
        return None

    if 'management' in val:
        return 'Management'
    if 'professional' in val:
        return 'Professional'
    if 'skilled manual' in val:
        return 'Skilled Manual'
    if 'manual' in val:
        return 'Manual'

    return None


In [44]:
df['marital_status'] = df['person_description'].apply(extract_marital)
df['gender'] = df['person_description'].apply(extract_gender)
df['num_children'] = df['person_description'].apply(extract_children)
df['education_level'] = df['person_description'].apply(extract_education)
df['job_type'] = df['person_description'].apply(extract_job)


In [45]:
df.drop(columns=['person_description'], inplace=True)

In [46]:
def parse_customer_order(val):
    if pd.isna(val):
        return pd.Series([None, None, None])

    pattern = r'^(.*?) from (.*?) department,\s*Ordered Brand\s*:\s*(.*)$'
    match = re.search(pattern, val)

    if match:
        product = match.group(1).strip()
        department = match.group(2).strip()
        brand = match.group(3).strip()
        return pd.Series([product, department, brand])

    return pd.Series([None, None, None])

df[['product_category', 'department', 'brand']] = (
    df['customer_order'].apply(parse_customer_order)
)


In [47]:
df.drop(columns=['customer_order'], inplace=True)

# **Exploration les données**

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28556 entries, 0 to 28555
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             28533 non-null  object 
 1   yearly_income                  28462 non-null  float64
 2   gross_weight                   23687 non-null  float64
 3   net_weight                     23727 non-null  float64
 4   is_recyclable                  28474 non-null  object 
 5   additional_features_in_market  25301 non-null  object 
 6   promotion_name                 24126 non-null  object 
 7   store_kind                     28556 non-null  object 
 8   store_sales                    28485 non-null  float64
 9   store_cost                     28484 non-null  float64
 10  grocery_area                   25859 non-null  float64
 11  frozen_area                    25969 non-null  float64
 12  meat_area                      25801 non-null 

In [49]:
df['store_area'].unique()

array([nan, '2572.86', '2200.59', ..., '1883.64', '2287.92', '1972.31'],
      dtype=object)

In [50]:
df['store_area'] = pd.to_numeric(
    df['store_area'],
    errors='coerce'
)


In [51]:
df['package_weight']=pd.to_numeric(df['package_weight'],errors='coerce')

In [52]:
df.shape

(28556, 33)

In [53]:
(df['store_area'] < 0).sum()


np.int64(8)

In [54]:
print((df['store_area'] < 0).sum())
print((df['grocery_area'] < 0).sum())
print((df['frozen_area'] < 0).sum())
print((df['meat_area'] < 0).sum())

8
12
8
8


In [55]:
cols_area = ['store_area', 'grocery_area', 'frozen_area', 'meat_area']

df[cols_area] = df[cols_area].where(df[cols_area] >= 0, np.nan)


In [56]:
df['package_weight']=df['gross_weight']-df['net_weight']

In [57]:
df.to_csv('final_train.csv',index=False)